# 라이브러리 

In [67]:
import os 
import warnings 
warnings.filterwarnings('ignore')
import cv2 
import pandas as pd 
from glob import glob 
from tqdm import tqdm 
import random 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from easydict import EasyDict as edict
import json 
import cv2 


import torch 
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models 

from utils import img_dset
from utils.Dataset import Data_loader,train_valid_split,Data_init
from utils.Augmentation import valid_augmenter,train_augmenter,dacon_augmenter
from utils.Trainer import epoch_run,valid_epoch_run
from utils import Model
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# Main

In [70]:
cfg = edict({
    'img_size' : 320,
    'batch_size' :16 ,
    'train_ratio' : 0.8, 
    'num_epochs' : 30, 
    'num_fold' : 5, 
    'model_name' : 'efficientnet_b4',
    'lr' : 1e-4, 
    'device' : 'cuda:0',
    'save_path' : './Save_models/skfold_effb4/',
    'seed' : 41 ,
    'crop_ratio' : 0.5 
})

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True   

def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score
    

def Save_record(record):
    save_record = pd.DataFrame(record)
    save_record.columns = ['fold','epoch_loss','acc','epoch_val_loss','val_acc']
    save_record.to_csv(f'{cfg.save_path}/record.csv',index=False)
    

seed_everything(cfg.seed) # Seed 고정
record = [] 
img_dirs,labels = Data_init('./Data')
for k,(train_index,valid_index) in enumerate(StratifiedKFold(n_splits=5).split(img_dirs,labels)):        
    #데이터 로드 
    if k == 0:
        label_encoder = {value:key for key,value in enumerate(np.unique(labels))}
    else:
        img_dirs, labels = Data_init('./Data')
    labels = pd.Series(labels).apply(lambda x : label_encoder[x]).values

    #Train - Valid split 
    train_img_dirs, valid_img_dirs = img_dirs[train_index],img_dirs[valid_index]
    train_labels, valid_labels = labels[train_index],labels[valid_index]
    
    #데이터셋, 데이터 로더 
    train_loader = Data_loader(train_img_dirs,train_labels,cfg,augmenter=dacon_augmenter(cfg))
    valid_loader = Data_loader(valid_img_dirs,valid_labels,cfg,augmenter=valid_augmenter(cfg),shuffle=False)
    
    #모델 config 
    model = Model(cfg.model_name,num_classes=len(np.unique(labels))).to(cfg.device)
    criterion = torch.nn.CrossEntropyLoss().to(cfg.device)
    optimizer = torch.optim.RAdam(model.parameters(),lr=cfg.lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=100,eta_min=0)
    scaler = torch.cuda.amp.GradScaler()
    
    print(f'Current Fold : {k}')
    best = 0
    for epoch in tqdm(range(cfg.num_epochs)):
        epoch_loss,acc = epoch_run(train_loader,model,cfg.device,optimizer,criterion,scheduler,scaler)
        epoch_loss_valid, acc_valid = valid_epoch_run(valid_loader,model,cfg.device,criterion)
        print('\nEpoch ', epoch)
        print(f'Train Loss : {epoch_loss} | Train F1 : {acc}')
        print(f'Valid Loss : {epoch_loss_valid} | Valid F1 : {acc_valid}')
        record.append([k,epoch_loss,acc,epoch_loss_valid,acc_valid])
        if epoch == 0:
            best = acc_valid 
            if os.path.exists(cfg.save_path) == False:
                os.mkdir(cfg.save_path)
            else:
                torch.save(model,cfg.save_path + f'{k}_fold_best.pt')
        else:
            if acc_valid > best:
                torch.save(model,cfg.save_path + f'{k}_fold_best.pt')
                print(f'Best_save{epoch}')

df = pd.DataFrame(cfg.values()).T
df.columns = cfg.keys() 
df.to_csv(f'{cfg.save_path}cfg.csv')
 

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b4_ra2_320-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_ra2_320-7eb33cd5.pth


Current Fold : 0


  0%|          | 0/30 [00:00<?, ?it/s]


Epoch  0
Train Loss : 3.7251438450168934 | Train F1 : 0.03173098684539334
Valid Loss : 3.2596336570945947 | Valid F1 : 0.05114597874988134


  3%|▎         | 1/30 [02:08<1:01:56, 128.15s/it]